In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import HeteroData
from torch_geometric.loader import NeighborLoader
from torch_geometric.transforms import ToUndirected
from torch_geometric.nn import SAGEConv, HeteroConv, GATv2Conv, GraphNorm
from torch.optim import Adam

import logging

import numpy as np
import os
import pandas as pd
import math

import gc

import matplotlib.pyplot as plt


import json
import pickle


import plotly.graph_objects as go



from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

import sys, os
sys.path.append(os.path.abspath(".."))

import similarity


# GNN

In [2]:
dir = './Data'
data = torch.load(f'{dir}/graph_data_embedded_v1_temporal_coded_s3.pt', weights_only=False)

## Functions

In [3]:
class QuantileLoss(nn.Module):
    def __init__(self, quantile=0.5):
        super().__init__()
        self.quantile = quantile

    def forward(self, preds, target):
        errors = target - preds
        return torch.max(
            (self.quantile - 1) * errors,
            self.quantile * errors
        ).mean()


class WeightedMSELoss(nn.Module):
    def __init__(self, threshold=10.0, high_weight=2.0):
        super().__init__()
        self.threshold = threshold
        self.high_weight = high_weight

    def forward(self, preds, target):
        weights = torch.ones_like(target)
        weights[target > self.threshold] = self.high_weight
        return (weights * (preds - target) ** 2).mean()


def get_loss_fn(loss_type="mse"):
    """
    loss_type options:
    - 'mse'
    - 'mae'
    - 'huber'
    - 'quantile'
    - 'weighted'
    """
    if loss_type == "mse":
        return nn.MSELoss()
    elif loss_type == "mae":
        return nn.L1Loss()
    elif loss_type == "huber":
        return nn.SmoothL1Loss(beta=1.0)
    elif loss_type == "quantile":
        return QuantileLoss(quantile=0.5)
    elif loss_type == "weighted":
        return WeightedMSELoss(threshold=10.0, high_weight=3.0)
    else:
        raise ValueError(f"Unknown loss type: {loss_type}")

In [4]:
def save_hyperparameters(hyperparameters, model_name, directory ='/Models'):

  model_dir = f'{directory}/{model_name}/'
  if not os.path.exists(model_dir):
      os.makedirs(model_dir)

  hyperparameters_path = os.path.join(model_dir, f'hyperparameters.json')

  with open(hyperparameters_path, 'w') as f:
      json.dump(hyperparameters, f)

In [5]:
def create_patient_stratified_masks(data, n_bins=10, test_size=0.1, val_size=0.1, seed=42):

    patient_to_stay_map = data['patient', 'HAS_STAY', 'stay'].edge_index.cpu()
    all_patient_ids = torch.unique(patient_to_stay_map[0]).tolist()

    patient_stay_df = pd.DataFrame({
        'patient_id': patient_to_stay_map[0].numpy(),
        'stay_idx': patient_to_stay_map[1].numpy(),
        'y': data['stay'].y[patient_to_stay_map[1]].cpu().numpy()
    })
    patient_mean = patient_stay_df.groupby('patient_id')['y'].mean().reset_index()
    patient_mean['bin'] = pd.qcut(patient_mean['y'], q=n_bins, duplicates='drop', labels=False)

    train_patients, temp_patients = train_test_split(
        patient_mean['patient_id'],
        test_size=test_size + val_size,
        stratify=patient_mean['bin'],
        random_state=seed
    )
    temp_df = patient_mean[patient_mean['patient_id'].isin(temp_patients)]
    val_patients, test_patients = train_test_split(
        temp_df['patient_id'],
        test_size=test_size / (test_size + val_size),
        stratify=temp_df['bin'],
        random_state=seed
    )

    train_set = set(train_patients)
    val_set = set(val_patients)
    test_set = set(test_patients)

    num_stays = data['stay'].num_nodes
    train_mask = torch.zeros(num_stays, dtype=torch.bool)
    val_mask = torch.zeros(num_stays, dtype=torch.bool)
    test_mask = torch.zeros(num_stays, dtype=torch.bool)

    for patient_id, stay_idx in zip(patient_to_stay_map[0], patient_to_stay_map[1]):
        pid = patient_id.item()
        sid = stay_idx.item()
        if pid in train_set:
            train_mask[sid] = True
        elif pid in val_set:
            val_mask[sid] = True
        elif pid in test_set:
            test_mask[sid] = True

    assert not (train_mask & val_mask).any()
    assert not (train_mask & test_mask).any()
    assert not (val_mask & test_mask).any()

    data['stay'].train_mask = train_mask
    data['stay'].val_mask = val_mask
    data['stay'].test_mask = test_mask

    print(f"Total stays: {num_stays}")
    print(f"Training stays: {train_mask.sum().item()}")
    print(f"Validation stays: {val_mask.sum().item()}")
    print(f"Test stays: {test_mask.sum().item()}")

    return data


In [6]:
import random

def make_similarity_v1(data, node_list, k_size):
    for node,k in zip(node_list, k_size):
        data = similarity.add_similarity_edges(data,node, k=k)
        print(f'{node}, done!')
    return data

def mak_similarity_v2(data, edge_list, edge_min, edgee_max):
    for edge in edge_list:
        k = random.randint(edge_min, edgee_max)
        new_edge_name = f'SIMILAR_BY_{edge[1][4:]}'
        print(f'{new_edge_name}, k:{k}')
        data = similarity.build_similarity_edges_from_edge_attr(
            data,
            edge,
            new_edge_name=new_edge_name,
            k=k
        )
        print('done')

    return data

In [7]:
class HeteroGNN(nn.Module):
  def __init__(self, data, metadata, hidden_channels=64, out_channels=1,
                conv_layer=GATv2Conv, dropout=0.3):
      super().__init__()
      self.hidden_channels = hidden_channels
      self.dropout = nn.Dropout(dropout)
      self.conv_layer = conv_layer

      self.lin_dict = nn.ModuleDict()
      self.res_lin_dict = nn.ModuleDict()
      self.norms = nn.ModuleDict()
      self.embeddings = nn.ModuleDict()

      for ntype in metadata[0]:
          if getattr(data[ntype], 'x', None) is not None:
              in_channels = data[ntype].x.size(1)
              self.lin_dict[ntype] = nn.Linear(in_channels, hidden_channels)
              self.res_lin_dict[ntype] = nn.Linear(in_channels, hidden_channels)
              self.norms[ntype] = GraphNorm(hidden_channels)
          else:
              self.embeddings[ntype] = nn.Embedding(data[ntype].num_nodes, hidden_channels)
              self.norms[ntype] = nn.Identity()

      self.conv1 = HeteroConv({
          et: conv_layer((-1, -1), hidden_channels, add_self_loops=False)
          for et in metadata[1]
      }, aggr='mean')
      self.conv2 = HeteroConv({
          et: conv_layer((-1, -1), hidden_channels, add_self_loops=False)
          for et in metadata[1]
      }, aggr='mean')

      self.reg_head = nn.Linear(hidden_channels, out_channels)

  def forward(self, x_dict, edge_index_dict, batch_dict=None):
        device = next(self.parameters()).device
        x_ready = {}

        # -------- Input projection per node-type --------
        for ntype in x_dict:
            x = x_dict[ntype]

            if x is None:
                # learn embedding for featureless nodes
                x = self.embeddings[ntype](
                    torch.arange(self.embeddings[ntype].num_embeddings, device=device)
                )
            else:
                h = self.lin_dict[ntype](x)
                res = self.res_lin_dict[ntype](x)
                x = F.relu(h + res)

            # normalization
            norm = self.norms[ntype]
            if isinstance(norm, GraphNorm) and batch_dict is not None and ntype in batch_dict:
                x = norm(x, batch_dict[ntype])
            else:
                x = norm(x)

            x_ready[ntype] = self.dropout(x)

        # -------- Layer 1 --------
        filtered_ei1 = {et: edge_index_dict[et] 
                        for et in self.conv1.convs.keys() 
                        if et in edge_index_dict}

        x1 = self.conv1(x_ready, filtered_ei1)
        x1 = {k: self.dropout(F.relu(v)) for k, v in x1.items()}

        # -------- Layer 2 (now actually used) --------
        filtered_ei2 = {et: edge_index_dict[et] 
                        for et in self.conv2.convs.keys() 
                        if et in edge_index_dict}

        x2 = self.conv2(x1, filtered_ei2)

        # -------- Residual skip from input --------
        x_final = {}
        for ntype in x2:
            h = x2[ntype]
            if ntype in x_ready:
                h = h + x_ready[ntype]
            x_final[ntype] = self.dropout(F.relu(h))

        # -------- Regression head on stay nodes --------
        return self.reg_head(x_final['stay'])





In [8]:
def train(model, loader, optimizer, loss_fn, device='cuda'):
    model.train()
    total_loss = 0
    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        pred = model(batch.x_dict, batch.edge_index_dict)
        target = batch['stay'].y.unsqueeze(1).to(device)
        loss = loss_fn(pred, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, cohen_kappa_score

BINS = [0,1,2,3,5,7,10,14,21,30,np.inf]

def msle(y_true, y_pred):
    return np.mean((np.log1p(y_true) - np.log1p(y_pred))**2)

def mad(y_true, y_pred):
    return np.median(np.abs(y_true - y_pred))

def log_mape(y_true, y_pred):
    return np.mean(np.abs((np.log1p(y_true) - np.log1p(y_pred)) / np.log1p(y_true))) * 100

def kappa_linear(y_true, y_pred):
    yb = np.digitize(y_true, BINS)
    pb = np.digitize(y_pred, BINS)
    return cohen_kappa_score(yb, pb, weights="linear")


@torch.no_grad()
def evaluate(model, loader, loss_fn, device='cuda'):
    model.eval()

    all_preds, all_targets = [], []
    total_loss = 0

    for batch in loader:
        batch = batch.to(device)

        pred = model(batch.x_dict, batch.edge_index_dict)
        target = batch['stay'].y.unsqueeze(1).to(device)

        loss = loss_fn(pred, target)
        total_loss += loss.item()

        all_preds.append(pred.cpu().numpy())
        all_targets.append(target.cpu().numpy())

    preds = np.concatenate(all_preds).flatten()
    targets = np.concatenate(all_targets).flatten()


    r2 = r2_score(targets, preds) 
    val_loss = total_loss / len(loader)

    true_los = np.expm1(targets)
    pred_los = np.expm1(preds)


    mae = mean_absolute_error(true_los, pred_los)
    mse = mean_squared_error(true_los, pred_los)
    msle_val = msle(true_los, pred_los)
    mad_val = mad(true_los, pred_los)
    lmape = log_mape(true_los, pred_los)
    kappa = kappa_linear(true_los, pred_los)

    return (
        val_loss,   # log-space
        r2,         # log-space
        mae,
        mse,
        msle_val,
        mad_val,
        lmape,
        kappa,
        preds,
        targets
    )




In [9]:
def train_loop(model, train_loader, val_loader, optimizer, loss_fn, epochs,
               patience, model_dir, device='cuda', plot_rate=10, start_epoch= 0):

    best_val_loss = float('inf')
    counter = 0

    for epoch in range(start_epoch + 1, epochs + start_epoch + 1):
        train_loss = train(model, train_loader, optimizer, loss_fn, device)
        val_loss, r2, mae, mse, msle, mad, lmape, kappa, preds, targets= evaluate(model, val_loader, loss_fn, device)

        train_losses.append(train_loss)
        val_losses.append(val_loss)
        val_r2_scores.append(r2)
        val_mae_scores.append(mae)
        val_mse_scores.append(mse)
        val_msle_scores.append(msle)
        val_mad_scores.append(mad)
        val_lmape_scores.append(lmape)
        val_kappa_scores.append(kappa)

        print(
                f"Epoch {epoch} | "
                f"Loss: {val_loss:.4f} | "
                f"R2(log): {r2:.4f} | "
                f"MAE: {mae:.3f} | "
                f"MSE: {mse:.3f} | "
                f"MSLE: {msle:.3f} | "
                f"MAD: {mad:.3f} | "
                f"logMAPE: {lmape:.2f}% | "
                f"Kappa: {kappa:.3f}"
                )


        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
            torch.save(model.state_dict(), f'{model_dir}')

        else:
            counter += 1
            if counter >= patience:
                print(f"Early stopping after {epoch} epochs due to no improvement in validation loss.")
                torch.save(model.state_dict(), f'{model_dir[:-3]}_best.pt')

                with open(f'{model_dir[:-8]}targets.pkl', 'wb') as f:
                  pickle.dump(targets, f)

                with open(f'{model_dir[:-8]}peredicts.pkl', 'wb') as f:
                  pickle.dump(preds, f)
                break
        torch.cuda.empty_cache()
        gc.collect()

        if epoch % plot_rate == 0 or epoch == 5:

            fig1 = go.Figure()

            fig1.add_trace(go.Scatter(
                x=list(range(1, len(train_losses) + 1)),
                y=train_losses,
                mode='lines',
                name='Train Loss'
            ))

            fig1.add_trace(go.Scatter(
                x=list(range(1, len(val_losses) + 1)),
                y=val_losses,
                mode='lines',
                name='Validation Loss'
            ))

            fig1.update_layout(
                title='Training and Validation Loss over Epochs',
                xaxis_title='Epoch',
                yaxis_title='Loss',
                template='plotly_white'
            )

            fig1.show()

            fig2 = go.Figure()

            fig2.add_trace(go.Scatter(
                x=list(range(1, len(val_r2_scores) + 1)),
                y=val_r2_scores,
                mode='lines',
                name='Validation R2 (log-space)'
            ))

            fig2.add_trace(go.Scatter(
                x=list(range(1, len(val_mae_scores) + 1)),
                y=val_mae_scores,
                mode='lines',
                name='MAE (days)'
            ))

            fig2.add_trace(go.Scatter(
                x=list(range(1, len(val_mse_scores) + 1)),
                y=val_mse_scores,
                mode='lines',
                name='MSE (days²)'
            ))

            fig2.add_trace(go.Scatter(
                x=list(range(1, len(val_msle_scores) + 1)),
                y=val_msle_scores,
                mode='lines',
                name='MSLE'
            ))

            fig2.add_trace(go.Scatter(
                x=list(range(1, len(val_mad_scores) + 1)),
                y=val_mad_scores,
                mode='lines',
                name='MAD (days)'
            ))

            fig2.add_trace(go.Scatter(
                x=list(range(1, len(val_lmape_scores) + 1)),
                y=val_lmape_scores,
                mode='lines',
                name='log-MAPE (%)'
            ))

            fig2.add_trace(go.Scatter(
                x=list(range(1, len(val_kappa_scores) + 1)),
                y=val_kappa_scores,
                mode='lines',
                name='Kappa'
            ))

            fig2.update_layout(
                title='Validation Metrics over Epochs',
                xaxis_title='Epoch',
                yaxis_title='Metric Value',
                template='plotly_white'
            )

            fig2.show()

 
            with open(f'{model_dir[:-8]}results.json', 'wb') as f:
                pickle.dump({
                    'train_losses': train_losses,
                    'val_losses': val_losses,
                    'val_r2_scores': val_r2_scores,
                    'val_mae_scores': val_mae_scores,
                    'val_mse_scores': val_mse_scores,
                    'val_msle_scores': val_msle_scores,
                    'val_mad_scores': val_mad_scores,
                    'val_lmape_scores': val_lmape_scores,
                    'val_kappa_scores': val_kappa_scores,
                    'best_val_loss': best_val_loss,
                }, f)


            with open(f'{model_dir[:-8]}targets.pkl', 'wb') as f:
                pickle.dump(targets, f)

            with open(f'{model_dir[:-8]}peredicts.pkl', 'wb') as f:
                pickle.dump(preds, f)


    return (
        train_losses,
        val_losses,
        val_r2_scores,
        best_val_loss,
        val_mae_scores,
        val_mse_scores,
        val_msle_scores,
        val_mad_scores,
        val_lmape_scores,
        val_kappa_scores
    )


In [10]:
def get_conv_layer(conv_layer):
    if conv_layer == 'SAGEConv':
        return SAGEConv
    elif conv_layer == 'GATv2Conv':
        return GATv2Conv
    elif conv_layer == 'HGTConv':
        return HGTConv
    else:
        raise ValueError(f"Unknown convolution layer: {conv_layer}")

In [11]:
def sanitize_hetero_(data):
    for (src, rel, dst), edge_index in list(data.edge_index_dict.items()):
        src_num = data[src].num_nodes
        dst_num = data[dst].num_nodes

        src_idx = edge_index[0]
        dst_idx = edge_index[1]

        mask = (src_idx < src_num) & (dst_idx < dst_num)
        if mask.sum() < edge_index.size(1):
            print(f"[sanitize] dropping {(~mask).sum().item()} edges from {(src, rel, dst)}")
        data[(src, rel, dst)].edge_index = edge_index[:, mask]


## Train

In [12]:
sanitize_hetero_(data)
data = create_patient_stratified_masks(data)

Total stays: 94432
Training stays: 75445
Validation stays: 9543
Test stays: 9444


In [13]:
data = similarity.add_similarity_edges(data, 'patient', k=15)
data = similarity.add_similarity_edges(data, 'diagnosis', k=15)
data = similarity.add_similarity_edges(data, 'procedure', k=10)
data = similarity.add_similarity_edges(data, 'prescriptions', k=5)

In [14]:
import random

event_list = [
      ('stay', 'HAS_INPUT', 'input_event'),
      ('stay', 'HAS_OUTPUT', 'output_event'),
      ('stay', 'HAS_WARNING', 'warning'),
      ('stay', 'HAS_TEXTED', 'text_event'),
      ('stay', 'HAS_CHARTED', 'chart_event'),
              ]

for edge in event_list:
  k = random.randint(30, 35)
  new_edge_name = f'SIMILAR_BY_{edge[1][4:]}'
  print(f'{new_edge_name}, k:{k}')
  data = similarity.build_similarity_edges_from_edge_attr(
      data,
      edge,
      new_edge_name=new_edge_name,
      k=k
  )
  print('done')


SIMILAR_BY_INPUT, k:34
done
SIMILAR_BY_OUTPUT, k:32
done
SIMILAR_BY_WARNING, k:35
done
SIMILAR_BY_TEXTED, k:31
done
SIMILAR_BY_CHARTED, k:35
done


In [16]:
torch.save(data, f'{dir}/graph_data_embedded_v1_temporal_coded_s3.pt')

In [12]:
metadata = data.metadata()
train_loader = NeighborLoader(
    data,
    num_neighbors={key: [20,25] for key in data.edge_types},
    batch_size=128,
    input_nodes=('stay', data['stay'].train_mask),
    shuffle=True,
)

val_loader = NeighborLoader(
    data,
    num_neighbors={key:  [20,25] for key in data.edge_types},
    batch_size=1024,
    input_nodes=('stay', data['stay'].val_mask),
    shuffle=False,
)

test_loader = NeighborLoader(
    data,
    num_neighbors={key:  [20,25] for key in data.edge_types},
    batch_size=1024,
    input_nodes=('stay', data['stay'].test_mask),
    shuffle=False,
)

In [13]:
hyperparameters = {
    'hidden_channels': 256,
    'conv_layer': 'GATv2Conv',
    'dropout': 0.3,
    'lr': 1e-3,
    'loss': 'huber',
    'data' : 'v1_v2_s2g',
    "similarity": 'v2',
    'model_name': 'model_v1_s2_s2g_fixed',
    'directed' : 'True',
    'num_neighbors' : [20,25]
}




model_dir = './Models'

model_name = hyperparameters['model_name']

save_hyperparameters(hyperparameters, model_name, directory = model_dir)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [14]:
model = HeteroGNN(data,
                  metadata,
                  hidden_channels=hyperparameters['hidden_channels'],
                  out_channels=1,
                  conv_layer=get_conv_layer(hyperparameters['conv_layer']),
                  dropout = hyperparameters['dropout']).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr= hyperparameters['lr'])
loss_fn = get_loss_fn(hyperparameters['loss'])


train_losses, val_losses, val_r2_scores, val_mae_scores, val_mse_scores = [], [], [], [], []
val_msle_scores, val_mad_scores, val_lmape_scores, val_kappa_scores = [], [], [], []
    

# model.load_state_dict(torch.load(f'{model_dir}/{model_name}/model.pt'))
# model.eval()

# with open(f'{model_dir}/{model_name}/results.json', 'rb') as f:
#     result = pickle.load(f)

# train_losses = result['train_losses']
# val_losses = result['val_losses']
# val_r2_scores = result['val_r2_scores']
# val_mae_scores = result['val_mae_scores']
# val_mse_scores = result['val_mse_scores']
start_epoch = len(train_losses)

total_epochs = 350
best_val_loss = float('inf')
epochs = total_epochs - start_epoch

patience = 50
counter = 0


train_losses, val_losses, val_r2_scores, best_val_loss, val_mae_scores, val_mse_scores, val_msle_scores, val_mad_scores, val_lmape_scores, val_kappa_scores = train_loop(model, train_loader, val_loader, 
                   optimizer, loss_fn, epochs,
                   patience, f'{model_dir}/{model_name}/model.pt',
                   device, plot_rate=50 ,start_epoch= start_epoch)


Epoch 1 | Loss: 0.1619 | R2(log): 0.3079 | MAE: 2.591 | MSE: 33.480 | MSLE: 0.349 | MAD: 1.161 | logMAPE: 43.22% | Kappa: 0.313
Epoch 2 | Loss: 0.1523 | R2(log): 0.3502 | MAE: 2.478 | MSE: 30.992 | MSLE: 0.327 | MAD: 1.120 | logMAPE: 41.65% | Kappa: 0.351
Epoch 3 | Loss: 0.1499 | R2(log): 0.3618 | MAE: 2.476 | MSE: 30.013 | MSLE: 0.321 | MAD: 1.120 | logMAPE: 41.37% | Kappa: 0.372
Epoch 4 | Loss: 0.1481 | R2(log): 0.3716 | MAE: 2.437 | MSE: 28.669 | MSLE: 0.317 | MAD: 1.193 | logMAPE: 42.48% | Kappa: 0.370
Epoch 5 | Loss: 0.1475 | R2(log): 0.3736 | MAE: 2.415 | MSE: 28.373 | MSLE: 0.315 | MAD: 1.187 | logMAPE: 42.30% | Kappa: 0.371


Epoch 6 | Loss: 0.1403 | R2(log): 0.4041 | MAE: 2.309 | MSE: 26.864 | MSLE: 0.300 | MAD: 1.145 | logMAPE: 41.28% | Kappa: 0.388
Epoch 7 | Loss: 0.1413 | R2(log): 0.4025 | MAE: 2.325 | MSE: 26.300 | MSLE: 0.301 | MAD: 1.188 | logMAPE: 42.28% | Kappa: 0.390
Epoch 8 | Loss: 0.1382 | R2(log): 0.4150 | MAE: 2.281 | MSE: 25.844 | MSLE: 0.295 | MAD: 1.156 | logMAPE: 41.61% | Kappa: 0.397
Epoch 9 | Loss: 0.1356 | R2(log): 0.4261 | MAE: 2.255 | MSE: 25.490 | MSLE: 0.289 | MAD: 1.139 | logMAPE: 41.40% | Kappa: 0.404
Epoch 10 | Loss: 0.1324 | R2(log): 0.4381 | MAE: 2.203 | MSE: 25.324 | MSLE: 0.283 | MAD: 1.085 | logMAPE: 40.38% | Kappa: 0.408
Epoch 11 | Loss: 0.1343 | R2(log): 0.4317 | MAE: 2.229 | MSE: 25.151 | MSLE: 0.286 | MAD: 1.129 | logMAPE: 41.34% | Kappa: 0.407
Epoch 12 | Loss: 0.1353 | R2(log): 0.4277 | MAE: 2.245 | MSE: 25.096 | MSLE: 0.288 | MAD: 1.159 | logMAPE: 41.79% | Kappa: 0.404
Epoch 13 | Loss: 0.1319 | R2(log): 0.4427 | MAE: 2.203 | MSE: 24.514 | MSLE: 0.281 | MAD: 1.123 | log

KeyboardInterrupt: 

In [28]:
test_loss, test_r2, test_mae, test_mse, test_msle, test_mad, test_lmape, test_kappa ,predicts, targets = evaluate(model, test_loader, loss_fn, device)

with open(f'{model_dir}/{model_name}/test_result.json', 'wb') as f:
    pickle.dump({'test_loss':test_loss,
              'test_r2': test_r2,
              'test_mae': test_mae,
              'test_mse': test_mse,
              'test_msle' : test_msle,
               'test_mad' : test_mad,
               'test_lmape' : test_lmape,
               'test_kappa' : test_kappa 
              }, f)

with open(f'{model_dir}/{model_name}/test_predicts.pkl', 'wb') as f:
    pickle.dump(predicts, f)

with open(f'{model_dir}/{model_name}/test_targets.pkl', 'wb') as f:
    pickle.dump(targets, f)

In [ ]:
print(f"Loss: {test_loss:.4f} | "
                f"R2(log): {test_r2:.4f} | "
                f"MAE: {test_mae:.3f} | "
                f"MSE: {test_mse:.3f} | "
                f"MSLE: {test_msle:.3f} | "
                f"MAD: {test_mad:.3f} | "
                f"logMAPE: {test_lmape:.2f}% | "
                f"Kappa: {test_kappa:.3f}"
                )

Loss: 0.1083 | R2(log): 0.5302 | MAE: 1.846 | MSE: 22.723 | MSLE: 0.236 | MAD: 0.689 | logMAPE: 30.31% | Kappa: 0.521
